In [5]:
pip install python-dotenv

In [7]:
import requests
import time
import json
import os
import sys
import re
from collections import defaultdict

In [8]:
api_key = 'KtRvYY7rwjXj7TJs9obbtABGr5pV7Gxn'

In [11]:
def fetch_articles(query, start_date, end_date, page=0):
    params = {
        'q': query,  # Substitua 'Netflix' com sua consulta de pesquisa
        'api-key': api_key,
        'begin_date': start_date,
        'end_date': end_date,
        'page': page
    }

    url = "https://api.nytimes.com/svc/search/v2/articlesearch.json"
    response = requests.get(url, params=params)

    print("Request URL:", response.url)
    print("Response Headers:", response.headers)

    if response.status_code != 200:
        print('Failed to fetch data:', response.status_code)
        sys.exit(1)

    return response.json()

In [13]:
url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'

# Dicionário para armazenar os artigos por ano e mês
articles_by_year_month_after2015 = defaultdict(list)

# Loop sobre cada ano e mês de 2015 a 2021
for year in range(2015, 2022):  # De 2015 a 2021
    for month in range(1, 13):  # Cada mês de janeiro (1) a dezembro (12)
        begin_date = f'{year}{month:02d}01'  # Primeiro dia do mês
        end_date = f'{year}{month:02d}28'  # Último dia do mês (para simplificar)

        # Parâmetros da API
        params = {
            'q': 'Netflix',  # Alterar para a palavra-chave desejada
            'api-key': api_key,
            'begin_date': begin_date,
            'end_date': end_date
        }

        # Fazer a requisição
        response = requests.get(url, params=params)
        print(f"Fetching articles from {begin_date} to {end_date}...")

        if response.status_code != 200:
            print('Failed to fetch data:', response.status_code)
            continue

        data = response.json()

        # Pegar os artigos do mês
        articles = data.get('response', {}).get('docs', [])

        # Armazenar os artigos no dicionário por ano e mês
        for article in articles:
            headline = article['headline']['main'] if 'headline' in article and 'main' in article['headline'] else 'No headline'
            web_url = article['web_url'] if 'web_url' in article else 'No URL'

            # Armazena no formato 'YYYYMM'
            articles_by_year_month_after2015[begin_date[:6]].append({
                'title': headline,
                'url': web_url
            })

        # Aguardar para evitar limite de taxa da API
        time.sleep(12)

# Exibir os artigos agrupados por ano e mês
for year_month, articles in articles_by_year_month_after2015.items():
    print(f"Articles from {year_month}:")
    for article in articles:
        print(f" - Title: {article['title']}")
        print(f"   URL: {article['url']}")
    print('---')


Fetching articles from 20150101 to 20150128...
Fetching articles from 20150201 to 20150228...
Fetching articles from 20150301 to 20150328...
Fetching articles from 20150401 to 20150428...
Fetching articles from 20150501 to 20150528...
Fetching articles from 20150601 to 20150628...
Fetching articles from 20150701 to 20150728...
Fetching articles from 20150801 to 20150828...
Fetching articles from 20150901 to 20150928...
Fetching articles from 20151001 to 20151028...
Fetching articles from 20151101 to 20151128...
Fetching articles from 20151201 to 20151228...
Fetching articles from 20160101 to 20160128...
Fetching articles from 20160201 to 20160228...
Fetching articles from 20160301 to 20160328...
Fetching articles from 20160401 to 20160428...
Fetching articles from 20160501 to 20160528...
Fetching articles from 20160601 to 20160628...
Fetching articles from 20160701 to 20160728...
Fetching articles from 20160801 to 20160828...
Fetching articles from 20160901 to 20160928...
Fetching arti

In [15]:
articles_by_year_month_after2015

defaultdict(list,
            {'201501': [{'title': 'Sundance 2015: Duplass Brothers, Netflix Make a Deal',
               'url': 'https://carpetbagger.blogs.nytimes.com/2015/01/24/sundance-2015-duplass-brothers-netflix-make-a-deal/'},
              {'title': 'Netflix Accelerates Ambitious Global Expansion as U.S. Growth Slows',
               'url': 'https://www.nytimes.com/2015/01/21/business/media/netflix-earnings.html'},
              {'title': 'Movie Studio by Amazon for Screens Big and Small',
               'url': 'https://www.nytimes.com/2015/01/20/business/amazon-to-start-theatrical-movie-operation-with-a-quick-streaming-tie-in.html'},
              {'title': 'As Content Pours From Everywhere, New Network Shows Strive for Boldness',
               'url': 'https://www.nytimes.com/2015/01/19/business/media/as-content-pours-from-everywhere-new-network-shows-strive-for-boldness.html'},
              {'title': 'Cleaning Up the Mac’s Printers List',
               'url': 'https://ww

In [16]:
articles

[{'title': 'Stream These 9 Titles Before They Leave Netflix in January',
  'url': 'https://www.nytimes.com/2021/12/27/arts/television/netflix-january.html'},
 {'title': 'Netflix and the Internet of Fads',
  'url': 'https://www.nytimes.com/2021/12/07/technology/netflix-tiktok-fads.html'},
 {'title': 'Review: ‘The Witcher,’ a Netflix Heavy Hitter, Finally Returns',
  'url': 'https://www.nytimes.com/2021/12/16/arts/television/review-the-witcher-netflix.html'},
 {'title': 'Netflix Holiday Movies Ranked, From Tree Toppers to Lumps of Coal',
  'url': 'https://www.nytimes.com/2021/12/03/movies/netflix-holiday-christmas.html'},
 {'title': 'Stream These 15 Titles Before They Leave Netflix in December',
  'url': 'https://www.nytimes.com/2021/12/01/arts/television/netflix-expiring-december.html'},
 {'title': 'The Best Movies and TV Shows New to Netflix, Amazon and Stan in Australia in December',
  'url': 'https://www.nytimes.com/2021/12/03/arts/television/december-netflix-australia-streaming-movi

In [24]:
import csv
# Nome do arquivo CSV que será criado
csv_file = 'articles.csv'

# Abrindo o arquivo CSV para escrita
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Escrevendo o cabeçalho (colunas)
    writer.writerow(['year_month', 'title'])

    # Percorrendo o dicionário e salvando cada artigo no CSV
    for year_month, articles in articles_by_year_month_after2015.items():
        for article in articles:
            title = article['title']
            writer.writerow([year_month, title])

print(f"Dados armazenados em {csv_file} com sucesso.")

Dados armazenados em articles.csv com sucesso.
